In [2]:
import sympy as sp
import numpy as np
import pprint
import os
import os.path
import pickle
sp.init_printing()

In [3]:
dim = (4,4)
N=143

In [4]:
ps = sp.symbols([f'p{i}' for i in range(dim[0])])
qs = sp.symbols([f'q{i}' for i in range(dim[1])])
ns = sp.symbols([f'n{i}' for i in range(len(bin(N))-2)])
symbol_arr = [ps, qs, ns]

symbol_arr

In [5]:
symbol_arr_factors = [sp.Add(*[sym*2**i for i,sym in enumerate(arr)]) for arr in symbol_arr] 

symbol_arr_factors


In [6]:
eq = (symbol_arr_factors[2]-symbol_arr_factors[0]*symbol_arr_factors[1])**2
eq

In [7]:
 def higher_to_second_order(self):
        # higher order to second order
        # there are no higher order c terms 
        # therefore we only consider combinations of p and q
        pq_combos = expand(Mul(*[sum(sym_list) for sym_list in [self.p,self.q]])).args



        # generate necessary t symbols for substitution
        self.t = symbols(' '.join([f't{i}' for i in range(len(pq_combos))]))

        # construct subs dict
        self.sub_dict = [[(pq, t_sym)] for pq, t_sym in zip(pq_combos,self.t) ]
       
        # for dynamic penalty weight give coeff 
        def construct_equality_constraint(pqt, coeff = 1):
            return (
                coeff* self.lamda*(pqt[0]*pqt[1]-2*pqt[0]*pqt[2]-2*pqt[1]*pqt[2]+3*pqt[2]) 
            )
        self.lower_order_expr = self.cost_expr.copy()


        constraints = []
        modified = True

        # run until no more modifiable terms
        while modified:
            modified = False
            modified_terms = []
            terms =[_arg for _arg in self.lower_order_expr.args]

            for term in terms:
                if (len(term.free_symbols) >= 3):
                    for sub_rule in self.sub_dict:
                        sub_term = term.subs(sub_rule)
                        # variable replacement starting
                        if sub_term != term:
                            pqt = list(sub_rule[0][0].args) + [sub_rule[0][1]]
                            constraints.append(construct_equality_constraint(pqt, coeff=abs(term.args[0])))
                            term = sub_term
                            modified = True # loop until no more modifications made
                            break
                
                modified_terms.append(term)
            self.lower_order_expr = Add(*modified_terms) 
        self.lower_order_expr += Add(*constraints) 

In [67]:

import numpy as np
import os
import os.path
import pickle

from sympy import *
import re
from dwave.system import DWaveSampler, EmbeddingComposite
import dimod
import pandas as pd
import pickle
import os
import os.path
import dwave.inspector

import json
from dimod.serialization.json import DimodEncoder, DimodDecoder

class ProblemC1Factor:
    def __init__(self, N):
        self.N = N
        self.lenN = len(bin(self.N))-2
        print(self.lenN)
        self.dim = (self.lenN-1, self.lenN //2)
        self.solver = ""



    def create_equation(self):
        self.p = symbols([f'p{i}' for i in range(self.dim[0])])
        self.q = symbols([f'q{i}' for i in range(self.dim[1])])
        self.n = symbols([f'n{i}' for i in range(self.lenN)])
        symbol_arr = [self.p, self.q, self.n]
    
        symbol_arr_factors = [sp.Add(*[sym*2**i for i,sym in enumerate(arr)]) for arr in symbol_arr] 

        self.cost_expr = (symbol_arr_factors[2]-symbol_arr_factors[0]*symbol_arr_factors[1])**2
        print(self.cost_expr)
        self.cost_expr = expand(self.cost_expr).subs([(term**2, term) for term in self.n + self.p + self.q])

        n_subs_dict = {self.n[i]:nbin for i,nbin in enumerate(bin(self.N)[2:][::-1])}
        print(n_subs_dict)
        self.cost_expr = self.cost_expr.subs(n_subs_dict)
    def higher_to_second_order(self):
        # higher order to second order
        # there are no higher order c terms 
        # therefore we only consider combinations of p and q
        pq_combos = expand(Mul(*[sum(sym_list) for sym_list in [self.p,self.q]])).args



        # generate necessary t symbols for substitution
        self.t = symbols(' '.join([f't{i}' for i in range(len(pq_combos))]))

        # construct subs dict
        self.sub_dict = [[(pq, t_sym)] for pq, t_sym in zip(pq_combos,self.t) ]
       
        # for dynamic penalty weight give coeff 
        def construct_equality_constraint(pqt, coeff = 1):
            return (
                coeff *(pqt[0]*pqt[1]-2*pqt[0]*pqt[2]-2*pqt[1]*pqt[2]+3*pqt[2]) 
            )
        self.lower_order_expr = self.cost_expr.copy()


        constraints = []
        modified = True

        # run until no more modifiable terms
        while modified:
            modified = False
            modified_terms = []
            terms =[_arg for _arg in self.lower_order_expr.args]

            for term in terms:
                if (len(term.free_symbols) >= 3):
                    for sub_rule in self.sub_dict:
                        sub_term = term.subs(sub_rule)
                        # variable replacement starting
                        if sub_term != term:
                            pqt = list(sub_rule[0][0].args) + [sub_rule[0][1]]
                            constraints.append(construct_equality_constraint(pqt, coeff=abs(term.args[0])))
                            term = sub_term
                            modified = True # loop until no more modifications made
                            break
                
                modified_terms.append(term)
            self.lower_order_expr = Add(*modified_terms) 
        self.lower_order_expr += Add(*constraints) 

    def to_qubo(self):
        # construct qubo with named variables s, remove offset 
        lower_order_coeff = self.lower_order_expr.as_coefficients_dict()
        offset = lower_order_coeff[1]
        qubo = {(tuple(str(k).split('*'))*2)[0:2] : v  for k,v in lower_order_coeff.items() if len(str(k).split('*')) < 3 and k != 1}
        return (qubo, offset)

    def generate_bqm(self):
        qubo, offset = self.to_qubo()
        self.bqm = dimod.BinaryQuadraticModel.from_qubo(Q = qubo, offset=offset)
        print(self.bqm)

    def init_solving(self):
            self.sampler = DWaveSampler(region='eu-central-1')  # alternativ um eine QPU in Nordamerika auszuwählen: sampler = DWaveSampler(region='na-west-1')
            self.solver = EmbeddingComposite(self.sampler)

    # Run bqms
    def run_get_success_rates(self, num_reads = 1000):
        if num_reads > 0:
            if not self.solver:
                self.init_solving()


            print(self.N)
            success_rates = {}
            results = {}
            valids = {}
            response = self.solver.sample(self.bqm, num_reads = num_reads, return_embedding=True)
            self.result = response.to_pandas_dataframe().sort_values(by='energy')

        
        p_columns = [f'p{i}' for i in range(len(self.p))]
        q_columns = [f'q{i}' for i in range(len(self.q))]

        p_df = self.result[p_columns]
        q_df = self.result[q_columns]

        self.result['a'] = p_df.apply(lambda row: int(''.join(list(row.values.astype(str)))[::-1],2), axis=1) 
        self.result['b'] = q_df.apply(lambda row: int(''.join(list(row.values.astype(str)))[::-1],2), axis=1)
    
        self.result['valida'] = self.result['a'].apply(lambda r: int(self.N)%int(r)==0 if int(r) != 0 else False)
        self.result['validb'] = self.result['b'].apply(lambda r: int(self.N)%int(r)==0 if int(r) != 0 else False)

       # self.result['valid'] = self.result['valida'].bool() or self.result['validb'].bool()
        #valid = self.result[(self.result['valid'] == True)]
        #success_rate = 0 if valid.empty  or num_reads == 0 else valid['num_occurrences'].sum()/ num_reads *100

        return (self.result)




In [68]:
P = ProblemC1Factor(N=3631381)
P.create_equation()
P.cost_expr
P.higher_to_second_order()
P.lower_order_expr
P.generate_bqm()
for _ in range(5):
    print(P.run_get_success_rates().query('valida or validb'))

22
(n0 + 2*n1 + 1024*n10 + 2048*n11 + 4096*n12 + 8192*n13 + 16384*n14 + 32768*n15 + 65536*n16 + 131072*n17 + 262144*n18 + 524288*n19 + 4*n2 + 1048576*n20 + 2097152*n21 + 8*n3 + 16*n4 + 32*n5 + 64*n6 + 128*n7 + 256*n8 + 512*n9 - (q0 + 2*q1 + 1024*q10 + 4*q2 + 8*q3 + 16*q4 + 32*q5 + 64*q6 + 128*q7 + 256*q8 + 512*q9)*(p0 + 2*p1 + 1024*p10 + 2048*p11 + 4096*p12 + 8192*p13 + 16384*p14 + 32768*p15 + 65536*p16 + 131072*p17 + 262144*p18 + 524288*p19 + 4*p2 + 1048576*p20 + 8*p3 + 16*p4 + 32*p5 + 64*p6 + 128*p7 + 256*p8 + 512*p9))**2
{n0: '1', n1: '0', n2: '1', n3: '0', n4: '1', n5: '0', n6: '0', n7: '0', n8: '1', n9: '0', n10: '0', n11: '1', n12: '0', n13: '1', n14: '1', n15: '0', n16: '1', n17: '1', n18: '1', n19: '0', n20: '1', n21: '1'}
BinaryQuadraticModel({'t154': 198821376.0, 't165': 392927232.0, 't176': 766992384.0, 't155': 799808832.0, 't187': 1458536448.0, 't166': 1590195840.0, 't198': 2615279616.0, 't177': 3142704384.0, 't220': 3218079744.0, 't209': 4023386112.0, 't188': 6134659584.0,

ValueError: no embedding found